Mowing Detection with Sentinel-2 & SAVI

In [ ]:
import ee
import geemap
import datetime

ee.Initialize()
Map = geemap.Map()


Define Area & Dates

In [12]:
roi = ee.Geometry.Rectangle([78.0, 12.8, 78.2, 13.0])  # Tambaram region
date1 = '2025-08-01'
date2 = '2025-08-15'


Load Sentinel-2 & Cloud Mask

In [13]:
def mask_clouds(image):
    cloud_prob = image.select('MSK_CLDPRB')
    mask = cloud_prob.lt(20)
    return image.updateMask(mask)

def get_s2_sr(date):
    return (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterBounds(roi)
            .filterDate(date, ee.Date(date).advance(10, 'day'))
            .map(mask_clouds)
            .median())


4. Compute SAVI

In [28]:
def compute_savi(image):
    nir = image.select('B8')
    red = image.select('B4')
    L = 0.5
    savi = nir.subtract(red).divide(nir.add(red).add(L)).multiply(1 + L)
    return savi.rename('SAVI')


In [27]:
img1 = compute_savi(get_s2_sr(date1))
img2 = compute_savi(get_s2_sr(date2))


In [26]:
savi_diff = img1.subtract(img2).rename('SAVI_Diff')


In [25]:
mowing_mask = savi_diff.gt(0.2)  # Threshold: >20% drop


6. Visualize

In [24]:
Map.centerObject(roi, 12)
Map.addLayer(savi_diff, {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'green']}, 'SAVI Difference')
Map.addLayer(mowing_mask.updateMask(mowing_mask), {'palette': ['yellow']}, 'Mowing Detected')
Map


Map(bottom=30731.0, center=[12.900005665978481, 78.09999999999982], controls=(WidgetControl(options=['position…

In [22]:
def detect_mowing(date1, date2, roi):
    img1 = compute_savi(get_s2_sr(date1))
    img2 = compute_savi(get_s2_sr(date2))
    savi_diff = img1.subtract(img2).rename('SAVI_Diff')
    mowing_mask = savi_diff.gt(0.2)
    
    Map.centerObject(roi, 12)
    Map.addLayer(savi_diff, {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'green']}, 'SAVI Difference')
    Map.addLayer(mowing_mask.updateMask(mowing_mask), {'palette': ['yellow']}, 'Mowing Detected')
    return Map


In [30]:
detect_mowing(date1, date2, roi)


Map(bottom=7781166.0, center=[12.900005665978481, 78.09999999999982], controls=(WidgetControl(options=['positi…